In [ ]:
f= read("federalistpapers.txt", String)

subEssay = String[]

i = 1
while i != nothing 
    next_occurrence_start = findnext("FEDERALIST No.", f, i)
    next_occurrence_end = findnext("PUBLIUS", f, i)
    println(SubString(f, next_occurrence_start[1], next_occurrence_end[2]))
    if next_occurrence_start == nothing
        break
    end
    i = next_occurrence_end[1] + 1
end


In [ ]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

println(essays)

# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        for j in eachindex(paragraphs)
            paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    editedText = replace(editedText, "\r\n" => " ")
    editedText = replace(editedText, r"[,:;'-]"=> "")
    return editedText
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

array = editParagraphs(array)

# Question 9
authorSet = ["Hamilton", "Madison", "Jay", "Unknown", "All"]

function addParagraphsByAuthor(paragraphs, authors)
    essaysByAuthor = [Array{String}[], Array{String}[], Array{String}[], Array{String}[], Array{String}[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
        elseif(cmp(authors[i], "MADISON") == 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            hamilton = Array{String}(undef, 0)
            madison = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(hamilton, paragraphs[i][j])
                push!(madison, paragraphs[i][j])
            end
            push!(essaysByAuthor[1], hamilton)
            push!(essaysByAuthor[2], madison)
        elseif(cmp(authors[i], "JAY") == 0)
            jay = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(jay, paragraphs[i][j])
            end
            push!(essaysByAuthor[3], jay)
        else
            unknown = Array{String}(undef, 0)
            for j in eachindex(paragraphs[i])
                push!(unknown, paragraphs[i][j])
            end
            push!(essaysByAuthor[4], unknown)
        end
    end
    essaysByAuthor[5] = paragraphs
    return essaysByAuthor
end

paragraphsByAuthor = addParagraphsByAuthor(array, authors)

# Question 9a
function avgWordsPerEssay(authorParagraphs)
    wordCount = 0
    for i in eachindex(authorParagraphs)
        for j in eachindex(authorParagraphs[i])
            wordCount += (count(" ", authorParagraphs[i][j]) + 1)
        end
    end
    return wordCount / length(authorParagraphs)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(paragraphsByAuthor[i]))")
end

#Incomplete
#=
function avgWordLength(authorEssays)
    wordCount = avgWordsPerEssay(authorEssays) * length(authorEssays)
end
=#

In [9]:
# Tester
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
println(length(essays))
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end

# println(essays[1])

85


In [53]:
# Minh's implementation

# Note: the number of the essay will be index no.

# Break it up into the individual essays. Each essay starts with "Federalist No. n." Within each essay you can skip everything up to and including "To the People of the State of New York:" and the footnotes after "PUBLIUS." You can also skip everything up to the start of the first essay and everything after the end of the last essay. (3 points)
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)

from = "To the People of the State of New York:"
to = "PUBLIUS" 

# Loop over each essay and extract the text in the essays
for (i, essay) in enumerate(essays)

    search_from = findfirst(from, essay)
    search_to = findfirst(to, essay)

    # Find the indices of from and to in the essay string
    start_index = search_from[1] + length(from)
    end_index = search_to[1] - 1
    
    # Extract the text between from and to
    extracted_text = essay[start_index:end_index]

    # Remove trailing new lines in the start and the end of the needed text
    extracted_text = rstrip(extracted_text)
    extracted_text = lstrip(extracted_text)

    # Print the extracted text
    # println(extracted_text)
    # println(i)

    # Replace with only the needed text from each essay
    essays[i] = extracted_text 
    
end
# println(typeof(essays[1]))
println(essays[1])



AFTER an unequivocal experience of the inefficacy of the subsisting
federal government, you are called upon to deliberate on a new
Constitution for the United States of America. The subject speaks its
own importance; comprehending in its consequences nothing less than the
existence of the UNION, the safety and welfare of the parts of which it
is composed, the fate of an empire in many respects the most interesting
in the world. It has been frequently remarked that it seems to have been
reserved to the people of this country, by their conduct and example,
to decide the important question, whether societies of men are really
capable or not of establishing good government from reflection and
choice, or whether they are forever destined to depend for their
political constitutions on accident and force. If there be any truth
in the remark, the crisis at which we are arrived may with propriety be
regarded as the era in which that decision is to be made; and a wrong
election of the part we sh

In [42]:
# Original string
str = "Hello, world!"

# Create a SubString{String} slice
substring = str[1:5]

# Print the SubString{String}
println(substring)  # prints "Hello"

# Modify the SubString{String}
substring[1] = 'h'

# Print the original string
println(str)  # prints "hello, world!"


Hello


LoadError: MethodError: no method matching setindex!(::String, ::Char, ::Int64)